In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pip

In [ ]:
Customer_Demo = pd.read_excel('CustomerDemographics.xlsx')
Customer_Behavior_Data =  pd.read_excel('CustomerBehaviorData.xlsx')
Credit_Consumption_Data = pd.read_excel('CreditConsumptionData.xlsx')

In [ ]:
# Get metadata using info() method
print("DataFrame info:")
print(Customer_Demo.info())

In [ ]:
# Get summary statistics using describe() method
print("DataFrame summary statistics:")
print(Customer_Demo.describe())

In [ ]:
# Get metadata using info() method
print("DataFrame info:")
print(Customer_Behavior_Data.info())

In [ ]:
# Get summary statistics using describe() method
print("DataFrame summary statistics:")
print(Customer_Behavior_Data.describe())

In [ ]:
Credit_Consumption_Data

In [ ]:
# Get metadata using info() method
print("DataFrame info:")
print(Credit_Consumption_Data.info())

In [ ]:
# Get summary statistics using describe() method
print("DataFrame summary statistics:")
print(Credit_Consumption_Data.describe())

In [ ]:
# Perform the merge
merged_table = pd.merge(Customer_Demo, Customer_Behavior_Data,on='ID')
merged_table = pd.merge(merged_table, Credit_Consumption_Data, on='ID')

In [ ]:
merged_table

In [ ]:
# Get metadata using info() method
print("DataFrame info:")
merged_table.info()

In [ ]:
# Get summary statistics using describe() method
print("DataFrame summary statistics:")
merged_table.describe()

In [ ]:
def continuous_var_summary( x ):
    
    # freq and missings
    n_total = x.shape[0]
    n_miss = x.isna().sum()
    perc_miss = n_miss * 100 / n_total
    
    # outliers - iqr
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    lc_iqr = q1 - 1.5 * iqr
    uc_iqr = q3 + 1.5 * iqr
    
    
    return pd.Series( [ x.dtype, x.nunique(), n_total, x.count(), n_miss, perc_miss,
                       x.sum(), x.mean(), x.std(), x.var(), 
                       lc_iqr, uc_iqr, 
                       x.min(), x.quantile(0.01), x.quantile(0.05), x.quantile(0.10), 
                       x.quantile(0.25), x.quantile(0.5), x.quantile(0.75), 
                       x.quantile(0.90), x.quantile(0.95), x.quantile(0.99), x.max() ], 
                     
                    index = ['dtype', 'cardinality', 'n_tot', 'n', 'nmiss', 'perc_miss',
                             'sum', 'mean', 'std', 'var',
                        'lc_iqr', 'uc_iqr',
                        'min', 'p1', 'p5', 'p10', 'p25', 'p50', 'p75', 'p90', 'p95', 'p99', 'max']) 

In [ ]:
# An utility function to create dummy variable
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], 
                                 prefix = colname, 
                                 drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

### EDA

In [ ]:
#check for null value
merged_table.isnull().any().sum()

In [ ]:
mis_val_col =['account_type' , 'gender' , 'Income' , 'region_code', 'Avg_days_between_transaction', 
'cc_cons_may', 'dc_cons_jun', 'cc_count_apr', 'personal_loan_closed', 'investment_3',
'debit_count_apr', 'loan_enq', 'emi_active']

In [ ]:
merged_table[mis_val_col].isnull().any().sum()

In [ ]:
# drop rows where the values are null
# merged_table = merged_table.dropna(subset=mis_val_col)

In [ ]:
#check for null value
merged_table.isnull().any().sum()

In [ ]:
merged_table.info()

In [ ]:
# merged_table.to_excel('output.xlsx', index=False)

In [ ]:
merged_table[mis_val_col]

In [ ]:
miss_int_col=['region_code', 'Avg_days_between_transaction' ,'cc_cons_may', 
              'dc_cons_jun', 'cc_count_apr', 'personal_loan_closed', 'investment_3',
              'debit_count_apr', 'emi_active' ]
miss_obj_col=['account_type', 'gender', 'Income','loan_enq']

In [ ]:
for i in miss_int_col:
    merged_table[i] = np.where(pd.isnull(merged_table[i]), 
                               merged_table[i].mean(),
                               merged_table[i])
for j in miss_obj_col:
    merged_table[j] = np.where(pd.isnull(merged_table[j]), 
                               merged_table[j].mode().values[0],
                               merged_table[j])

In [ ]:
merged_table.info()

In [ ]:
num_col = merged_table.select_dtypes(include=['int','float']).columns

### One hot encoding

In [ ]:
obj_var = merged_table.select_dtypes(include=['object']).columns
obj_var

In [ ]:
merged_table['account_type'].value_counts()

In [ ]:
merged_table['gender'].value_counts()

In [ ]:
merged_table['Income'].value_counts()

In [ ]:
merged_table['loan_enq'].value_counts()

In [ ]:
# lets drop loan_enq as its cardinality is lowest
# and lets drop ID as its cardinality is very very high
merged_table.drop(columns=['ID','loan_enq'] , inplace=True)

In [ ]:
merged_table.shape

In [ ]:
merged_table = create_dummies(merged_table,['account_type','gender','Income'])

In [ ]:
merged_table.shape

#### Outlier treatment

In [ ]:
# separate the numeric and object variables
num_var = merged_table.select_dtypes(['float64', 'int64'])

In [ ]:
# get the data audit report for all the variables
num_var.apply(continuous_var_summary)

In [ ]:
num_var

In [ ]:
# Determine the number of rows and columns for the subplot grid
num_rows = 12
num_cols = 4

# Create the subplot grid
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 40))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Iterate over each numeric column and create a box plot
for i, column in enumerate(num_var.columns):
    # Select the corresponding axis for the current box plot
    ax = axes[i]
    
    # Create the box plot for the current column
    ax.boxplot(num_var[column])
    
    # Set the title of the box plot to the column name
    ax.set_title(column)
    
    # Remove x-axis ticks and labels for better visualization
    ax.set_xticks([])
    ax.set_xticklabels([])
    
    # Adjust the subplot spacing
    fig.subplots_adjust(hspace=0.5)
    
# Remove any extra subplots if the number of columns is not divisible by the number of rows
if len(num_var.columns) < num_rows * num_cols:
    for j in range(len(num_var.columns), num_rows * num_cols):
        fig.delaxes(axes[j])

# Display the plot
plt.show()


In [ ]:
# maximum age cannot be more than 100
merged_table.age.max()

In [ ]:
merged_table.age.quantile(0.99)

In [ ]:
# clealy there are many attributes which require outlier treatment

outl_col= ['age', 'Emp_Tenure_Years', 'cc_cons_apr','dc_cons_apr', 
          'cc_cons_may','dc_cons_may','cc_cons_jun','dc_cons_jun','cc_count_apr', 
          'cc_count_may','cc_count_jun','dc_count_apr', 'dc_count_may',
          'card_lim','investment_1', 'investment_2','investment_3','investment_4',
          'debit_amount_apr','credit_amount_apr','debit_count_apr',
          'credit_count_apr', 'max_credit_amount_apr','debit_amount_may',
          'credit_amount_may','credit_count_may','debit_count_may',
          'max_credit_amount_may', 'debit_amount_jun','credit_amount_jun',
          'credit_count_jun','max_credit_amount_jun', 'emi_active']        

In [ ]:
# This method is not working as the outliers are more than expected.

#for i in outl_col:
#    merged_table[i].clip(lower = merged_table[i].quantile(0.01),
#                         upper = merged_table[i].quantile(0.99), inplace=True)

In [ ]:
# Set the multiplier to adjust the IQR threshold
multiplier = 1.5

# Iterate over each column in the DataFrame
for col in outl_col:
# Calculate the first quartile (Q1) and third quartile (Q3) of the column
    Q1 = merged_table[col].quantile(0.25)
    Q3 = merged_table[col].quantile(0.75)
    
# Calculate the IQR (Interquartile Range)
    IQR = Q3 - Q1
    
# Define the lower and upper bounds for outliers
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
# Identify and treat outliers by capping them within the lower and upper bounds
    merged_table[col] = merged_table[col].apply(lambda x: lower_bound if 
                                  x < lower_bound else (upper_bound if 
                                                        x > upper_bound else x))

In [ ]:
# Create an 11x4 subplot grid
fig, axes = plt.subplots(11, 4, figsize=(15, 30))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Iterate over each numeric column and create a box plot
for i, column in enumerate(outl_col):
    # Select the corresponding axis for the current box plot
    ax = axes[i]
    
    # Create the box plot for the current column
    ax.boxplot(merged_table[column])
    
    # Set the title of the box plot to the column name
    ax.set_title(column)
    
    # Remove x-axis ticks and labels for better visualization
    ax.set_xticks([])
    ax.set_xticklabels([])
    
    # Adjust the subplot spacing
    fig.subplots_adjust(hspace=0.4)
    
# Remove any extra subplots if the number of columns is not divisible by 4
if len(outl_col) < 44:
    for j in range(len(outl_col), 44):
        fig.delaxes(axes[j])

# Display the plot
plt.show()


#### Now lets do the feature selection using Ridge and Lasso regression

In [ ]:
merged_table.columns

In [ ]:
merged_table

In [ ]:
# load the ML libraries
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

In [ ]:
merged_table['cc_cons'].isnull()

In [ ]:
Non_null_df = merged_table[~merged_table['cc_cons'].isnull()]
validation_df= merged_table[merged_table['cc_cons'].isnull()]

In [ ]:
y = Non_null_df.loc[:, Non_null_df.columns == 'cc_cons']
X = Non_null_df.loc[:, Non_null_df.columns != 'cc_cons']

In [ ]:
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#
## Assuming 'X' is your feature matrix and 'y' is your target variable
#
## Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#
## Create the Decision Tree Regressor model
#tree_reg = DecisionTreeRegressor()
#
## Train the model
#tree_reg.fit(X_train, y_train)
#
## Make predictions on the test set
#y_pred = tree_reg.predict(X_test)
#
## Calculate the Mean Squared Error
#mse = mean_squared_error(y_test, y_pred)
#print("Mean Squared Error:", mse)

In [ ]:
#### Clarly Decision tree regressor is not giving a good MSE 
# so let go with linear regression

In [ ]:
# Before linear regression we will do feature reduction using
# using ridge and lasso regression

In [ ]:
from sklearn import preprocessing

#scale all column of data, this will give a numpy array
x_scaled = preprocessing.scale(X)
x_scaled = pd.DataFrame(x_scaled, columns= X.columns)

y_scaled = preprocessing.scale(y)
y_scaled = pd.DataFrame(y_scaled, columns= y.columns)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=1)

In [ ]:
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)


In [ ]:
# Build a regression model

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
for idx, col_name in enumerate(X_train.columns):
    print("The coeficient for {} is {}".format(col_name, lin_reg.coef_[0][idx]))

In [ ]:
intercept = lin_reg.intercept_[0]

print('The intercept of our model using linear regression is {}'.format(intercept))

### Ridge Regression

In [ ]:
ridge = Ridge(alpha = .3) 
#coefficients are prevented to be too big by this alpha value
ridge.fit(X_train, y_train)

In [ ]:
for i,col in enumerate(X_train.columns):
    print("Ridge model coefficient for {} is :{}".format(col, ridge.coef_[0][i]))

In [ ]:
r_intercept = ridge.intercept_[0]
print('The intercept of our model using ridge regression is {}'.format(r_intercept))

### Lasso Regression

In [ ]:
lasso = Lasso(alpha = .1) 
#coefficients are prevented to be too big by this alpha value
lasso.fit(X_train, y_train)

In [ ]:
for i,col in enumerate(X_train.columns):
    print("Lasso model coefficient for {} is :{}".format(col, lasso.coef_[i]))

In [ ]:
# Build a regression model

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
for idx, col_name in enumerate(X_train.columns):
    print("The coeficient for {} is {}".format(col_name, lin_reg.coef_[0][idx]))

In [ ]:
intercept = lin_reg.intercept_[0]

print('The intercept of our model using linear regression is {}'.format(intercept))

### Ridge Regression

In [ ]:
ridge = Ridge(alpha = .3) 
#coefficients are prevented to be too big by this alpha value
ridge.fit(X_train, y_train)

In [ ]:
for i,col in enumerate(X_train.columns):
    print("Ridge model coefficient for {} is :{}".format(col, ridge.coef_[0][i]))

In [ ]:
r_intercept = ridge.intercept_[0]
print('The intercept of our model using ridge regression is {}'.format(r_intercept))

### Lasso Regression

In [ ]:
lasso = Lasso(alpha = .1) 
#coefficients are prevented to be too big by this alpha value
lasso.fit(X_train, y_train)

In [ ]:
for i,col in enumerate(X_train.columns):
    print("Lasso model coefficient for {} is :{}".format(col, lasso.coef_[i]))